In [ ]:
#''''''''''''''''''''''''''''''''''''''''''''
# Vandit Jyotindra Gajjar
# ID: a1779153
# Introduction-to-Statistical-Machine-Learning
# Assignment: 3
#''''''''''''''''''''''''''''''''''''''''''''

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import io
from collections import Counter
train = pd.read_csv(''/content/drive/My Drive/Colab Notebooks/ISML_a3/mnist_train.csv'').to_numpy()
test = pd.read_csv(''/content/drive/My Drive/Colab Notebooks/ISML_a3/mnist_test.csv'').to_numpy()

In [ ]:
X_train = train[:,1:]
y_train = train[:,0]
X_test = test[:,1:]
y_test = test[:,0]

In [ ]:
def standardize(X):
  standardizedArray = np.zeros(shape=np.shape(X))
  tempArray = np.zeros(np.shape(X)[1])

  for i in range(np.shape(X)[1]):
    mean = np.mean(X[:, i])
    std = np.std(X[:, i])
    if std == 0:
      std = 1
    tempArray = np.empty(0)
    #print('mean: ', mean, 'std: ', std)
    
    for element in X[:, i]:
      tempArray = np.append(tempArray, ((element - mean) / std))
    
    standardizedArray[:,i] = tempArray

  return standardizedArray

def projectionMatrix(eigen_vectors, num):
  projection_matrix = (eigen_vectors.T[:][:num]).T
  return projection_matrix

In [ ]:
X_std = standardize(X_train)
Xtest_std = standardize(X_test)

In [ ]:
print('Cov')
X_covariance_matrix = np.cov(X_std.T)
Xtest_covariance_matrix = np.cov(Xtest_std.T)

print('Eigen')
X_eigen_values, X_eigen_vectors = np.linalg.eig(X_covariance_matrix)
Xtest_eigen_values, Xtest_eigen_vectors = np.linalg.eig(Xtest_covariance_matrix)

In [7]:
"""# Calculating the explained variance on each of componentsvariance_explained = []
variance_explained = []

for i in X_eigen_values:
     variance_explained.append((i/sum(X_eigen_values))*100)
        
print(variance_explained)"""
X_projection_matrix = projectionMatrix(X_eigen_vectors, 10)
print(np.shape(X_projection_matrix))
X_PCA = X_std.dot(X_projection_matrix)

In [ ]:
class KNearestNeighbor(object):
  """ a kNN classifier with L2 distance """

  def __init__(self):
    pass

  def train(self, X, y):
    self.X_train, self.y_train = X, y
    
  def predict(self, X, k=1):
    return self.predict_labels(self.compute_distances(X), k=k)

  def compute_distances(self, X):
    dists = np.zeros((X.shape[0], self.X_train.shape[0])) 
    dists = np.sqrt(np.sum(X**2, axis=1).reshape(num_test, 1) + np.sum(self.X_train**2, axis=1) - 2 * X.dot(self.X_train.T))
    return dists

  def predict_labels(self, dists, k=1):
    y_pred = np.zeros(dists.shape[0])
    for i in range(num_test):
      closest_y = []
      top_k_indx = np.argsort(dists[i])[:k]
      closest_y = self.y_train[top_k_indx]
      vote = Counter(closest_y)
      count = vote.most_common()
      y_pred[i] = count[0][0]
    return y_pred

In [9]:
classifier = KNearestNeighbor()

for i in range(10, 257):
  X_projection_matrix = projectionMatrix(X_eigen_vectors, i)
  X_PCA = X_std.dot(X_projection_matrix)
  Xtest_projection_matrix = projectionMatrix(Xtest_eigen_vectors, i)
  Xtest_PCA = Xtest_std.dot(Xtest_projection_matrix)
  classifier.train(X_PCA, y_train)
  distance = classifier.compute_distances(Xtest_PCA)
  y_test_pred = classifier.predict_labels(distance, k=1)
  num_correct = np.sum(y_test_pred == y_test)
  num_test = distance.shape[0]
  accuracy = float(num_correct) / num_test
  print('For '+ str(i) + ' dimensions we', 'got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))



In [10]:
#without PCA
distance2 = distanceMetrics(X_train, X_test)

In [11]:
accuracy = 0
for i in range(len(distance2)):
  if y_train[distance2[i].index(min(distance2[i]))] == y_test[i]:
    accuracy += 1
  #print(y_train[distance2[i].index(min(distance2[i]))], y_test[i])
accuracy = accuracy/len(distance2)
print(accuracy)

In [12]:
classifier = KNearestNeighbor()
classifier.train(X_train, y_train)
distance = classifier.compute_distances(X_test)

demension 10 selected, PCA and KNN

In [ ]:
y_test_pred = classifier.predict_labels(distance, k=1)

num_correct = np.sum(y_test_pred == y_test)
num_test = distance.shape[0]
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

In [ ]:
if __name__ == "__main__":   
    train_data = pd.read_csv("Data/usps.csv", header = None)
    test_data = pd.read_csv("Data/usps.t.csv", header = None)
    train_X = np.asarray(train_data.drop([0], axis = 1))
    train_y = np.asarray(train_data[0])
    test_X = np.asarray(test_data.drop([0], axis = 1))
    test_y = np.asarray(test_data[0])

    print('Training data has', train_X.shape[0],'samples and', train_X.shape[1],'features.',
          '\nTesting data has', test_X.shape[0],'samples and', test_X.shape[1],'features.')
    
    model = KNeighborsClassifier()
    model.fit(train_X,train_y)
    print("\nModel Accuracy before dimension reduction: ",model.score(test_X, test_y))
    
    
    reduced_dim = 64 
    print("\nReducing dimensions from ",train_X.shape[1], " to ",reduced_dim)
    
    # PCA
    X_train_transformed, mean_mat, projection = pca_fit_transform(train_X, k = reduced_dim)
    X_test_transformed = pca_transform(test_X, mean_mat, projection)
    model.fit(X_train_transformed,train_y)
    print("\nModel Accuracy after PCA dimension reduction: ",model.score(X_test_transformed, test_y))

In [ ]:
def add_n(X):
    temp = np.asarray(X)
    noise = np.random.randn(X.shape[0], X.shape[1])
    temp = np.hstack((temp, noise))
    X = pd.DataFrame(temp)
    return X

X_train = add_n(X_train)
X_test = add_n(X_test)
print(X_train.shape, X_test.shape)

In [ ]:
from sklearn.model_selection  import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def score(x, y):
    knn = KNeighborsClassifier(n_neighbors=10)
    scores = cross_val_score(knn, x, y[0], cv=6, scoring='accuracy')
    return scores

def eval(x, y):
    scorelist = []
    for i in x:
        s = score(i, y)
        scorelist.append(1-s.mean())

    plt.plot([i for i in range(len(scorelist), 0, -1)],scorelist)
    plt.title('error curve of keeping different features')
    plt.xlabel('features')
    plt.ylabel('Error')
    plt.show()   

train_list = []
for i in range(256, 10, -1):
    # training set PCA processing
    X_new = PCA(X_train, i, plot=False, info=False)
    # test set PCA processing
    Xt_new = PCA(X_test, i, plot=False, info=False)
    train_list.append(X_new)

eval(train_list, Y_train)

In [27]:
def pca_fit_transform(X, k = 1):
    mean_matrix = np.mean(X, axis = 0)
    X = X - mean_matrix              
    covariance_matrix = np.cov(X.T) 
    eigen_values, eigen_vectors= np.linalg.eig(covariance_matrix)  
    eigen_pairs = [(eigen_values[i], eigen_vectors[:,i]) for i in range(eigen_values.shape[0])] 
    eigen_pairs.sort(key=lambda x: x[0], reverse=True)  

    P = eigen_pairs[0][1].reshape(eigen_pairs[0][1].shape[0],1)
    for i in range(1,k):
        P = np.hstack((P,eigen_pairs[i][1].reshape(eigen_pairs[i][1].shape[0],1)))
    transformed = X.dot(P)                  
    return transformed, mean_matrix, P

In [ ]:
def pca_transform(X_test, mean_matrix, P):
    X_test = X_test - mean_matrix               
    transformed = X_test.dot(P)                  
    return transformed

In [28]:
def euclDistance(vector1,vector2):
    return np.sqrt(np.sum(pow(vector2-vector1,2)))

In [29]:
model = SVC(kernel='linear')
scorelist = []
i = 1
while True:
    if i >= X_train.shape[0]:
        break
    model.fit(X_train.iloc[i:i+100],Y_train.iloc[i:i+100])
    predict = model.predict(X_test)
    score = accuracy_score(predict, Y_test)
    scorelist.append(score)
    i += 100

In [ ]:
def plotF(ErrorRateList, title='unk', xlabel='unk', ylabel='unk', ylim=0.1):
    length = len(ErrorRateList)
    numIterList = [x for x in range(length)]
    plt.plot(numIterList, ErrorRateList, 'ro-')
    plt.xlim(0, length)
    plt.ylim(0, ylim)
    plt.xticks([x for x in range(length) if x%10 == 0], [x for x in numIterList ], rotation=0)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()